z < b => e=1(event indicator), we calculate the probability of surviving, this mean in case of event happening, probability of surviving should be low => true y shold be zero 

In [1]:
import tensorflow 
print(tensorflow.__version__)
import pandas as pd
from sksurv.metrics import concordance_index_censored
from sksurv.metrics import brier_score
import numpy as np
from lifelines.utils import concordance_index
from lifelines import KaplanMeierFitter
from pycox.evaluation import EvalSurv

2.0.0


In [2]:
tr_data = pd.read_csv('C:\\Users\\raibe\\Desktop\\Thesis Code\\DRSA\\data\\ndb_p\\10\\train_yzb.txt', header=None, sep=" ")
test_data = pd.read_csv('C:\\Users\\raibe\\Desktop\\Thesis Code\\DRSA\\data\\ndb_p\\10\\test_yzb.txt', header=None, sep=" ")
#print("MAX_sequence_length: {}".format(max(max(tr_data[2]),max(test_data[2])))

In [3]:
#tr_data.head()
print(max(tr_data[1]))
print(max(tr_data[2]))
print(max(test_data[1]))
print(max(test_data[2]))

146
140
146
126


In [4]:
test_o = pd.read_csv('C:\\Users\\raibe\\Desktop\\Thesis Code\\DRSA\\data\\ndb_p\\10\\testDRSA.txt', header=None)
print('number of batches: {}'.format(test_o[26] [test_o[26] == 0].count()))

number of batches: 102


In [5]:
fet_o = pd.read_csv('C:\\Users\\raibe\\Desktop\\Thesis Code\\DRSA\\data\\ndb_p\\10\\feat_ind.txt', header=None, sep="\t")
print('MAX_DEN: {}'.format(max(fet_o[1])))

MAX_DEN: 912736


In [ ]:
#max(test_o[25])

In [6]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import sys
from sklearn.metrics import roc_auc_score
import random
from tensorflow.python.ops import tensor_array_ops, control_flow_ops
from lifelines.utils import concordance_index
import os
import time
import datetime
import pdb
import signal
import math
TRAING_TIME = 15
SHUFFLE = True
LOAD_LITTLE_DATA = False

class SparseData(): # it returns arrays of the data, shuffled

    def shuffle(self):
        if SHUFFLE:
            np.random.shuffle(self.index)
        return self.data[self.index], self.seqlen[self.index], self.labels[self.index], self.market_price[self.index]

    def __init__(self, INPUT_FILE, win, all, discount): # INPUT_FILE: data file name, 
                                                        #all: true if the entire the dataset will be included, in case of doing lose, we need to get the entire data
        self.data = []
        self.labels = []
        self.seqlen = []
        self.market_price = []
        fi = open(INPUT_FILE, 'r')
        COUNT = 1
        max_d = -1
        self.finish_epoch = False
        for line in fi:
            if COUNT > 10000 and LOAD_LITTLE_DATA: # raiber: check if you are using the small version of the data
                break
            COUNT += 1
            s = line.split(' ') # raiber: take each line (type str) and put it in S
            #line = "0 42 21 0:1 66:1 124:1 127:1 138:1 144:1 147:1 150:1 285:1 419:1 679:1 780:1 924:1 1011:1 1479:1"
            slen = len(s) # raiber: get the length of the features, here 18
            t_indices = [] # [0, 66, 124, 127, 138, 144, 147, 150, 285, 419, 679, 780, 924, 1011, 1479]
            for i in range(3, slen): # starting from the 3rd index, the 0, 1, 2 indexes has y, z, b respectively 
                w = s[i].split(':') #  x is the list of features (multi-hot encoded as feat_id:1).
                td = int(w[0])
                t_indices.append(td)
                max_d = max(td, max_d)
            market_price = int(s[1]) # z
            bid_price = int(s[2]) # b
            # notice that the market price is a varibale taking the "z" value and also defined at the beginning as array to contain the entire "z" values, please change the name of one of them 
            if all: # they go over the entire dataset, creating the feature array dataset (data variable) by appending each line t_indices to it 
                                                                    # observed time array (seqlen)  by appending each line "b" to it
                                                                    # ture time array (market_price) by appending each line "z" true time to it
                                                                    # if b <= z then the lable would be [1., 0.] otherwise would be [0., 1.]
                if bid_price <= market_price: # if b <= z (survival zuncensored 
                    self.data.append(t_indices)  # data only conclude indices use this to get embedding
                    self.seqlen.append(int(bid_price / discount))
                    self.market_price.append(int(market_price / discount))
                    self.labels.append([1., 0.])  # so far we always lose, it means we still survial
                else: # if b > z
                    self.data.append(t_indices)  # data only conclude indices use this to get embedding
                    self.seqlen.append(int(bid_price / discount))
                    self.market_price.append(int(market_price / discount))
                    self.labels.append([0., 1.])  # we win means we dead

            else: # if not all (all = False)
                if bid_price <= market_price:  # if b <= z
                    if not win:# if lose (survive)
                        self.data.append(t_indices)  # data only conclude indices use this to get embedding
                        #bid_price= 3
                        #market_price = 2
                        self.seqlen.append(int(bid_price / discount))
                        self.market_price.append(int(market_price / discount))
                        self.labels.append([1., 0.])  # so far we always lose, it means we still survial
                else: # if b > z if our bid price (b) is bigger that the ture price we win
                    if win: # dead 
                        #bid_price = 3
                        #market_price = 2
                        self.data.append(t_indices)  # data only conclude indices use this to get embedding
                        self.seqlen.append(int(bid_price / discount))
                        self.market_price.append(int(market_price / discount))
                        self.labels.append([0., 1.])  # we win means we dead

        self.max_d = max_d
        fi.close()
        self.size = len(self.data)
        self.data = np.array(self.data)
        self.labels = np.array(self.labels)
        self.seqlen = np.array(self.seqlen)
        self.market_price = np.array(self.market_price)
        print("data size ", self.size, "\n")
        self.index = list(range(0, self.size)) # raiber modefied code to work with python 3.
        self.data, self.seqlen, self.labels, self.market_price = self.shuffle() #  it shuffles the indxes 
        self.batch_id = 0


    def next(self, batch_size): # returns a batch from data 
        if self.batch_id + batch_size > len(self.data):
            self.data, self.seqlen, self.labels, self.market_price = self.shuffle()
            self.batch_id = 0
            self.finish_epoch = True
        batch_data = self.data[self.batch_id:self.batch_id + batch_size]
        batch_labels = self.labels[self.batch_id:self.batch_id + batch_size]
        batch_seqlen = self.seqlen[self.batch_id:self.batch_id + batch_size]
        batch_market_price = self.market_price[self.batch_id:self.batch_id + batch_size]
        self.batch_id = self.batch_id + batch_size
        return np.array(batch_data), np.array(batch_labels), np.array(batch_seqlen), np.array(batch_market_price)
        

class biSparseData():
    def __init__(self, INPUT_FILE, discount):
        random.seed(time.time())
        #SparseData(INPUT_FILE, win, all, discount)
        #self.winData.size = 12044, self.loseData.size = 39703, this is for the train data, loseData is the entier data 
        self.winData = SparseData(INPUT_FILE, True, False, discount) # only win data, uncensored 
        self.loseData = SparseData(INPUT_FILE, False, True, discount)#todo lose data get all data, even the win = False here, has no effect because the data
        self.size = self.winData.size + self.loseData.size
    def next(self, batch):
        #win = int(random.random() * 100) % 11 == 1# todoe 1/10 get windata
        win = int(random.random() * 100) % 11 <= 5 # int(random.random() * 100) % 11 gives values between 0 and 10,
                                                      # the expression <= 5, will give us either true or false, 
                                                      # it is like take 50 % chance to get ture or false 
                                                      # which means randomly choose based on 50 probability from either win or lose data 
                                                      # again lose data contain the data
        if win:
            a, b, c, d = self.winData.next(batch)
            return a, b, c, d, True
        else:
            a, b, c, d = self.loseData.next(batch)
            return a, b, c, d, False


class BASE_RNN():

    train_data = None
    def init_matrix(self, shape):
        return tf.random.normal(shape, stddev=0.1) # Outputs random values from a normal distribution.
        # shape:	A 1-D integer Tensor or Python array. The shape of the output tensor.
        # stddev: standard deviation 

    def __init__(self,  EMB_DIM = 32,
                        FEATURE_SIZE = 13,
                        BATCH_SIZE = 128,
                        MAX_DEN = 1580000,
                        MAX_SEQ_LEN = 350,
                        TRAING_STEPS = 100000,
                        STATE_SIZE = 64,
                        LR = 0.001,
                        GRAD_CLIP = 5.0,
                        L2_NORM = 0.001,
                        INPUT_FILE = "2259",
                        ALPHA = 1.0,
                        BETA = 0.2,
                        ADD_TIME_FEATURE=False,
                        MIDDLE_FEATURE_SIZE = 30,
                        LOG_FILE_NAME=None,
                        FIND_PARAMETER = False,
                        SAVE_LOG=True,
                        OPEN_TEST=True,
                        ONLY_TRAIN_ANLP=False,
                        LOG_PREFIX="",
                        TEST_FREQUENT=True, # changed by raibrt to true 
                        ANLP_LR = 0.001,
                        DNN_MODEL = False,
                        QRNN_MODEL = False,
                        GLOAL_STEP = 0,
                        COV_SIZE = 1,
                        DOUBLE_QRNN = False,
                        ANLP_ROUND_ROBIN_RATE = 0.2,
                        DISCOUNT = 1
):
        self.DISCOUNT = DISCOUNT
        self.DOUBLE_QRNN = DOUBLE_QRNN
        self.ANLP_ROUND_ROBIN_RATE = ANLP_ROUND_ROBIN_RATE
        self.QRNN_MODEL = QRNN_MODEL
        self.global_step = GLOAL_STEP
        self.DNN_MODEL = DNN_MODEL
        self.ANLP_LR = ANLP_LR
        self.TEST_FREQUENT = TEST_FREQUENT
        self.ONLY_TRAIN_ANLP = ONLY_TRAIN_ANLP
        self.FIND_PARAMETER = FIND_PARAMETER
        self.add_time_feature = ADD_TIME_FEATURE
        self.MIDDLE_FEATURE_SIZE = MIDDLE_FEATURE_SIZE
        tf.compat.v1.reset_default_graph()
        self.TRAING_STEPS = TRAING_STEPS
        self.BATCH_SIZE = BATCH_SIZE
        self.STATE_SIZE = STATE_SIZE
        self.EMB_DIM = EMB_DIM
        self.FEATURE_SIZE = FEATURE_SIZE
        self.MAX_DEN = MAX_DEN
        self.MAX_SEQ_LEN = int(MAX_SEQ_LEN / self.DISCOUNT) + 10 # was changed by raiber to int to work with python 3
        self.LR = LR
        self.GRAD_CLIP = GRAD_CLIP
        self.L2_NORM = L2_NORM
        self.ALPHA = ALPHA
        self.BETA = BETA
        self.INPUT_FILE = INPUT_FILE # 2997
        self.SAVE_LOG = SAVE_LOG
        self.TRAIN_FILE = "C:\\Users\\raibe\\Desktop\\Thesis Code\\DRSA\\data\\ndb_p\\10\\train_yzb.txt"
        self.TEST_FILE = "C:\\Users\\raibe\\Desktop\\Thesis Code\\DRSA\\data\\ndb_p\\10\\test_yzb.txt"
        self.OPEN_TEST = OPEN_TEST # true
        self.COV_SIZE = COV_SIZE # 1

  

        para = None
        if LOG_FILE_NAME != None: # LOG_FILE_NAME is none 
            para = LOG_FILE_NAME
        else:
            para = LOG_PREFIX + str(self.EMB_DIM) + "_" + \
                str(BATCH_SIZE) + "_" + \
                str(self.STATE_SIZE) + "_" + \
                "{:.6f}".format(self.LR) + "_" + "{:.6f}".format(self.ANLP_LR) + "_" + \
                "{:.6f}".format(self.L2_NORM) + "_" + \
                INPUT_FILE + "_" + \
                "{:.2f}".format(self.ALPHA) + "_" \
                "{:.2f}".format(self.BETA) + "_" + str(ADD_TIME_FEATURE) + \
                   "_" + str(self.QRNN_MODEL) + "_" + str(self.COV_SIZE) + "_" + str(DISCOUNT)
        print (para, '\n')
        self.filename = para
        self.train_log_txt_filename = "C:\\Users\\raibe\\Desktop\\Thesis Code\\DRSA\\results\\ndb_p\\10\\" + para + '.train.log.txt'
        if os.path.exists(self.train_log_txt_filename):
            self.exist = True
        else:
            if self.SAVE_LOG:
                self.exist = False
                self.train_log_txt = open(self.train_log_txt_filename, 'w')
                self.train_log_txt.close()

    def get_survival_data(self, model, sess):
        alltestdata = SparseData(self.TEST_FILE, True, True) #all test data
        ret = []
        while alltestdata.finish_epoch == False:
            test_batch_x, test_batch_y, test_batch_len, test_batch_market_price = alltestdata.next(self.BATCH_SIZE)
            bid_loss, bid_test_prob, anlp, preds = sess.run(
                [self.cost, self.predict, self.anlp_node, self.preds],
                feed_dict={self.tf_x: test_batch_x,
                           self.tf_y: test_batch_y,
                           self.tf_bid_len: test_batch_len,
                           self.tf_market_price: test_batch_market_price
                           })
            ret.append(preds)
        return ret # return will have the preds 


    def load_data(self):
        self.train_data = biSparseData(self.TRAIN_FILE, self.DISCOUNT)
        self.test_data_win = SparseData(self.TEST_FILE, True, False, self.DISCOUNT) # win, only win test data 
        self.test_data_lose = SparseData(self.TEST_FILE, False, False, self.DISCOUNT) #lose, only lose test data

    def is_exist(self):
        if self.SAVE_LOG == False:
            return False
        return self.exist


    def create_graph(self):
        BATCH_SIZE = self.BATCH_SIZE
        self.tf_x = tf.compat.v1.placeholder(tf.int32, [BATCH_SIZE, self.FEATURE_SIZE], name="tf_x")
        self.tf_y = tf.compat.v1.placeholder(tf.float32, [BATCH_SIZE, 2], name="tf_y")
        self.tf_bid_len = tf.compat.v1.placeholder(tf.int32, [BATCH_SIZE], name="tf_len")
        self.tf_market_price = tf.compat.v1.placeholder(tf.int32, [BATCH_SIZE], name="tf_market_price")
        self.tf_control_parameter = tf.compat.v1.placeholder(tf.float32, [2], name="tf_control_parameter")
        alpha = self.tf_control_parameter[0]
        beta = self.tf_control_parameter[1]
        self.tf_rnn_len = tf.maximum(self.tf_bid_len, self.tf_market_price) + 2 # the longest time between b and z plus 2 
        embeddings = tf.Variable(self.init_matrix([self.MAX_DEN, self.EMB_DIM]))
        x_emds = tf.nn.embedding_lookup(params=embeddings, ids=self.tf_x)
        input = tf.reshape(x_emds, [BATCH_SIZE, self.FEATURE_SIZE * self.EMB_DIM])
        input_x = None
        if self.add_time_feature: # which is true 
            middle_layer = tf.compat.v1.layers.dense(input, self.MIDDLE_FEATURE_SIZE, tf.nn.relu)  # hidden layer

            def add_time(x):
                y = tf.reshape(tf.tile(x, [self.MAX_SEQ_LEN]), [self.MAX_SEQ_LEN, self.MIDDLE_FEATURE_SIZE])
                t = tf.reshape(tf.range(self.MAX_SEQ_LEN), [self.MAX_SEQ_LEN, 1])
                z = tf.concat([y, tf.cast(t, dtype=tf.float32)], 1)
                return z

            input_x = tf.map_fn(add_time, middle_layer)

        preds = None

        if self.DNN_MODEL:
            outlist = []
            for i in range(0, self.BATCH_SIZE):
                sigleout = tf.compat.v1.layers.dense(input_x[i], 1, tf.nn.sigmoid)
                outlist.append(sigleout)
            preds = tf.reshape(tf.stack(outlist, axis=0), [self.BATCH_SIZE, self.MAX_SEQ_LEN], name="preds")
        else:
            # input_x = tf.reshape(tf.tile(input, [1, self.MAX_SEQ_LEN]), [BATCH_SIZE, self.MAX_SEQ_LEN, self.FEATURE_SIZE * self.EMB_DIM])
            rnn_cell = None
            rnn_cell = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(num_units=self.STATE_SIZE)


            outputs, (h_c, h_n) = tf.compat.v1.nn.dynamic_rnn(
                rnn_cell,                   # cell you have chosen
                input_x,                    # input
                initial_state=None,         # the initial hidden state
                dtype=tf.float32,           # must given if set initial_state = None
                time_major=False,           # False: (batch, time step, input); True: (time step, batch, input)
                sequence_length=self.tf_rnn_len
            )

            new_output = tf.reshape(outputs, [self.MAX_SEQ_LEN * BATCH_SIZE, self.STATE_SIZE])

            with tf.compat.v1.variable_scope('softmax'):
                W = tf.compat.v1.get_variable('W', [self.STATE_SIZE, 1])
                b = tf.compat.v1.get_variable('b', [1], initializer=tf.compat.v1.constant_initializer(0))

            logits = tf.matmul(new_output, W) + b
            preds = tf.transpose(a=tf.nn.sigmoid(logits, name="preds"), name="preds")[0]


        self.preds = preds
        survival_rate = preds 
        death_rate = tf.subtract(tf.constant(1.0, dtype=tf.float32), survival_rate)
        batch_rnn_survival_rate = tf.reshape(survival_rate, [BATCH_SIZE, self.MAX_SEQ_LEN])
        batch_rnn_death_rate = tf.reshape(death_rate, [BATCH_SIZE, self.MAX_SEQ_LEN])
        
        # this line added by raiber, because I want the survival rate at every single time, from 1 to self.MAX_SEQ_LEN
        # the rate _result below is a multipication of different survival rate from time 1 to the last time, so the survival time at last tiem only 
        self.survival_rate_different_times = batch_rnn_survival_rate
        self.death_rate_different_times = batch_rnn_death_rate

        map_parameter = tf.concat([batch_rnn_survival_rate,
                                   tf.cast(tf.reshape(self.tf_bid_len, [BATCH_SIZE, 1]), tf.float32)],
                                  1)
        map_parameter = tf.concat([map_parameter,
                                   tf.cast(tf.reshape(self.tf_market_price, [BATCH_SIZE, 1]), tf.float32)],
                                  1)
        def reduce_mul(x):
            bid_len = tf.cast(x[self.MAX_SEQ_LEN], dtype=tf.int32)
            market_len = tf.cast(x[self.MAX_SEQ_LEN + 1], dtype=tf.int32)
            # survival_rate_last_one is the survival rate at the obsevation time, which is bid_length
            # we are muliplying the survival rates at each single time from 1 to observation time 
            survival_rate_last_one = tf.reduce_prod(input_tensor=x[0:bid_len])
            anlp_rate_last_one = tf.reduce_prod(input_tensor=x[0:market_len + 1])
            anlp_rate_last_two = tf.reduce_prod(input_tensor=x[0:market_len])
            ret = tf.stack([survival_rate_last_one, anlp_rate_last_one, anlp_rate_last_two])
            return ret

        self.mp_para = map_parameter
        rate_result = tf.map_fn(reduce_mul, elems=map_parameter ,name="rate_result")
        self.rate_result = rate_result
        log_minus = tf.math.log(tf.add(tf.transpose(a=rate_result)[2] - tf.transpose(a=rate_result)[1], 1e-20))#todo debug


        self.anlp_node = -tf.reduce_sum(input_tensor=log_minus) / self.BATCH_SIZE #todo load name
        self.anlp_node = tf.add(self.anlp_node, 0, name="anlp_node")
        self.final_survival_rate = tf.transpose(a=rate_result)[0]
        final_dead_rate = tf.subtract(tf.constant(1.0, dtype=tf.float32), self.final_survival_rate)

        self.predict = tf.transpose(a=tf.stack([self.final_survival_rate, final_dead_rate]), name="predict")
        cross_entropy = -tf.reduce_sum(input_tensor=self.tf_y*tf.math.log(tf.clip_by_value(self.predict,1e-10,1.0))) # tf.clip_by_value
                                                                                                    #this operation returns a tensor of the same type and shape as t 
                                                                                                    #with its values clipped to clip_value_min and clip_value_max. 
                                                                                                    #Any values less than clip_value_min are set to clip_value_min. 
                                                                                                    #Any values greater than clip_value_max are set to clip_value_max.

        tvars = tf.compat.v1.trainable_variables()
        lossL2 = tf.add_n([ tf.nn.l2_loss(v) for v in tvars ]) * self.L2_NORM
        cost = tf.add(cross_entropy, lossL2, name = "cost")  / self.BATCH_SIZE
        self.cost = tf.add(cost, 0, name="cost")
        optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=self.LR, beta2=0.99)#.minimize(cost)
        optimizer_anlp = tf.compat.v1.train.AdamOptimizer(learning_rate=self.ANLP_LR, beta2=0.99)#.minimize(cost)

        grads, _ = tf.clip_by_global_norm(tf.gradients(ys=self.cost, xs=tvars),
                                          self.GRAD_CLIP,
                                          )
        self.train_op = optimizer.apply_gradients(zip(grads, tvars), name="train_op")
        tf.compat.v1.add_to_collection('train_op', self.train_op)

        anlp_grads, _ = tf.clip_by_global_norm(tf.gradients(ys=self.anlp_node, xs=tvars),
                                          self.GRAD_CLIP,
                                          )
        self.anlp_train_op = optimizer_anlp.apply_gradients(zip(anlp_grads, tvars), name="anlp_train_op")
        tf.compat.v1.add_to_collection('anlp_train_op', self.anlp_train_op)


        self.com_cost = tf.add(alpha * self.cost, beta * self.anlp_node)
        com_grads, _ = tf.clip_by_global_norm(tf.gradients(ys=self.com_cost, xs=tvars),
                                          self.GRAD_CLIP,
                                          )

        self.com_train_op = optimizer.apply_gradients(zip(com_grads, tvars), name="train_op")
        tf.compat.v1.add_to_collection('com_train_op', self.com_train_op)

        correct_pred = tf.equal(tf.argmax(input=self.predict, axis=1), tf.argmax(input=self.tf_y, axis=1))
        self.accuracy = tf.reduce_mean(input_tensor=tf.cast(correct_pred, tf.float32), name="accuracy")


    def train_test(self,sess):
        self.load_data()
        init = tf.compat.v1.global_variables_initializer()
        self.sess = sess
        sess.run(init)
        saver = tf.compat.v1.train.Saver(max_to_keep=100)
        self.saver = saver
        TRAIN_LOG_STEP = int((self.train_data.size * 0.1) / self.BATCH_SIZE)
        train_auc_arr = []
        train_anlp_arr = []
        train_loss_arr = []
        train_auc_label = []
        train_death_label = []#added by raiber
        train_auc_prob = []
        train_time = [] #added by raiber 
        train_m_time = [] #added by raiber 
        total_train_duration = 0
        total_test_duration = 0
        TEST_COUNT = 0
        max_auc = -1
        min_anlp = 200
        enough_test = 0
        last_loss = [9999.0, 9999.0]
        start_time = time.time()
        
        #added by raiber
        self.max_valid = -99
        self.stop_flag = 0
        
        for step in range(1, self.TRAING_STEPS + 1):
            if self.stop_flag > 5: #for faster early stopping
                break
            self.global_step = step
            batch_x, batch_y, batch_len, batch_market_price, win = self.train_data.next(self.BATCH_SIZE)
            if self.ONLY_TRAIN_ANLP:
                if win: #if win
                    _, train_anlp, train_loss, train_outputs = sess.run([self.com_train_op, self.anlp_node, self.cost, self.predict],
                                                                    feed_dict={self.tf_x: batch_x,
                                                                             self.tf_y: batch_y,
                                                                             self.tf_bid_len: batch_len,
                                                                             self.tf_market_price:batch_market_price,
                                                                             self.tf_control_parameter:[self.ALPHA, self.BETA]
                                                       })
                    train_anlp_arr.append(train_anlp)
                    train_loss_arr.append(train_loss)
                    train_auc_label.append(batch_y.T[0])
                    train_auc_prob.append(np.array(train_outputs).T[0])
                else:
                    train_loss, train_outputs = sess.run([self.cost, self.predict], feed_dict={self.tf_x: batch_x,
                                                       self.tf_y: batch_y,
                                                       self.tf_bid_len: batch_len,
                                                       self.tf_market_price:batch_market_price,
                                                       self.tf_control_parameter:[self.ALPHA, self.BETA]
                                                       })
                    #print train_outputs
                    train_loss_arr.append(train_loss)
                    train_auc_label.append(batch_y.T[0])
                    train_auc_prob.append(np.array(train_outputs).T[0])
            else:
                if win: #if win
                    _, train_anlp, train_loss, train_outputs, preds= sess.run([self.com_train_op, self.anlp_node, self.cost, self.predict, self.preds],
                                                                    feed_dict={self.tf_x: batch_x,
                                                                             self.tf_y: batch_y,
                                                                             self.tf_bid_len: batch_len,
                                                                             self.tf_market_price:batch_market_price,
                                                                             self.tf_control_parameter:[self.ALPHA, self.BETA]
                                                       })
                    train_anlp_arr.append(train_anlp)
                    train_loss_arr.append(train_loss)
                    train_auc_label.append(batch_y.T[0])
                    train_death_label.append(batch_y.T[1])
                    train_auc_prob.append(np.array(train_outputs).T[0])
                    train_time.append(batch_market_price)
                    train_m_time.append(batch_market_price)
                else:
                    _, train_loss, train_outputs = sess.run([self.train_op, self.cost, self.predict], feed_dict={self.tf_x: batch_x,
                                                       self.tf_y: batch_y,
                                                       self.tf_bid_len: batch_len,
                                                       self.tf_market_price:batch_market_price,
                                                       self.tf_control_parameter:[self.ALPHA, self.BETA]
                                                       })
                    #print train_outputs
                    train_loss_arr.append(train_loss) 
                    train_auc_label.append(batch_y.T[0]) # in the first loop ww will have 100 list (number of steps), each one with 128 elements (batch size)
                    train_death_label.append(batch_y.T[1])
                    train_auc_prob.append(np.array(train_outputs).T[0])
                    train_time.append(batch_len)
                    train_m_time.append(batch_market_price)

            if step % 100 == 0:
                mean_anlp = np.array(train_anlp_arr[-99:]).mean()
                mean_loss = np.array(train_loss_arr[-99:]).mean()
                mean_auc = 0.0001
                if not self.ONLY_TRAIN_ANLP:
                    try:
                        mean_auc = roc_auc_score(np.reshape(train_auc_label, [1, -1])[0], np.reshape(train_auc_prob, [1, -1])[0])
                    except Exception:
                        print("AUC ERROE")
                        continue

                log = self.getStatStr("TRAIN", self.global_step, mean_auc, mean_loss, mean_anlp)
                print(log)
                self.force_write(log)
                train_loss_arr = []
                train_anlp_arr = []
                train_auc_label = []
                train_auc_prob = []
                if self.TEST_FREQUENT:
                    self.run_test(sess)
                    #self.save_model()

            #if self.global_step < 1000: # raiber changed from 300 to 1000, just for experimenting 
            #    if self.global_step % 100 == 0:
            #        self.run_test(sess, tr_time = train_time, tr_label = train_auc_label)
                    #self.save_model()
            #elif self.global_step < 2000:
            #    if self.global_step % 500 == 0:
            #        self.run_test(sess, tr_time = train_time, tr_label = train_auc_label)
            #        #self.save_model()
            #elif self.global_step < 10000:
            #    if self.global_step % 2000 == 0:
            #        self.run_test(sess, tr_time = train_time, tr_label = train_auc_label)
                    #self.save_model()
            #elif self.global_step <= 50000:
            #    if self.global_step % 2000 == 0:
            #        self.run_test(sess, tr_time = train_time, tr_label = train_auc_label)
                    #self.save_model()
            #else:
            #    break

    def run_model(self):
        config = tf.compat.v1.ConfigProto()
        config.gpu_options.allow_growth = True
        with tf.compat.v1.Session(config=config) as sess:
            self.train_test(sess)

    def save_model(self):
        print("model name: ", self.filename, " ", self.global_step, "\n")
        self.saver.save(self.sess, "C:\\Users\\raibe\\Desktop\\Thesis Code\\DRSA\\results\\ndb_p\\10\\saved_model\\model" + self.filename, global_step=self.global_step)

    def getStatStr(self, category ,step, mean_auc, mean_loss, mean_anlp):
        statistics_log = str(self.INPUT_FILE) + "\t" + category + "\t" + str(step) + "\t" \
                         "{:.6f}".format(mean_loss) + "\t" + \
                         "{:.4f}".format(mean_auc) + "\t" + \
                         "{:.4f}".format(mean_anlp) + "\t" + \
                         "{:.4f}".format(self.ALPHA * mean_loss + self.BETA * mean_anlp) + \
                         str(self.EMB_DIM) + "\t" + str(self.BATCH_SIZE) + "\t" + \
                         str(self.STATE_SIZE) + "\t" + \
                         "{:.6f}".format(self.LR) + "\t" + \
                         "{:.6f}".format(self.ANLP_LR) + "\t" + \
                         "{:.6}".format(self.L2_NORM) + "\t" +\
                         str(self.ALPHA) + '\t' + \
                         str(self.BETA) + "\n"
        return statistics_log

    def getStatStr_test(self, category ,step, mean_auc, mean_lc, mean_br, mean_loss, mean_anlp):
        statistics_log = str(self.INPUT_FILE) + "\t" + category + "\t" + str(step) + "\t" \
                         "{:.6f}".format(mean_loss) + "\t" + \
                         "{:.4f}".format(mean_auc) + "\t" + \
                         "{:.4f}".format(mean_lc) + "\t" + \
                         "{:.4f}".format(mean_br) + "\t" + \
                         "{:.4f}".format(mean_anlp) + "\t" + \
                         "{:.4f}".format(self.ALPHA * mean_loss + self.BETA * mean_anlp) + \
                         str(self.EMB_DIM) + "\t" + str(self.BATCH_SIZE) + "\t" + \
                         str(self.STATE_SIZE) + "\t" + \
                         "{:.6f}".format(self.LR) + "\t" + \
                         "{:.6f}".format(self.ANLP_LR) + "\t" + \
                         "{:.6}".format(self.L2_NORM) + "\t" +\
                         str(self.ALPHA) + '\t' + \
                         str(self.BETA) + "\n"
        return statistics_log

    def load(self, meta, ckpt, step): # this returns sess (session)
        tf.compat.v1.reset_default_graph()
        config = tf.compat.v1.ConfigProto()
        config.gpu_options.allow_growth = True
        saver = tf.compat.v1.train.import_meta_graph(meta)
        #self.load_data()
        self.global_step = step
        #with tf.Session(config=config) as sess:
        sess = tf.compat.v1.Session(config=config)
        saver.restore(sess, ckpt)
        graph = tf.compat.v1.get_default_graph()
        self.tf_x = graph.get_tensor_by_name("tf_x:0")
        self.tf_y = graph.get_tensor_by_name("tf_y:0")
        self.tf_bid_len = graph.get_tensor_by_name("tf_len:0")
        self.tf_market_price = graph.get_tensor_by_name("tf_market_price:0")
        self.accuracy = graph.get_tensor_by_name("accuracy:0")
        self.cost = graph.get_tensor_by_name("cost:0")
        self.predict = graph.get_tensor_by_name("predict:0")
        self.anlp_node = graph.get_tensor_by_name("anlp_node:0")
        self.train_op = tf.compat.v1.get_collection('train_op')[0]

        #self.anlp_train_op = graph.get_collection("anlp_train_op")[0]
        #self.train _op = graph.get_tensor_by_name("train_op:0")
        self.preds = graph.get_tensor_by_name("preds:0")
        #self.com_train_op = tf.get_collection("com_train_op")[0]
        #self.tf_control_parameter = graph.get_tensor_by_name("tf_control_parameter:0")
        # self.train_log_txt.write(statistics_log)
        return sess

    def run_test(self, sess):
        auc_arr = []
        s_t_sr = []
        s_t_dr = []
        cind_arr = []#added by raiber
        m_br_arr = []#added by raiber
        br_arr = []#added by raiber
        lc_arr = []#added by raiber
        surv_cind_arr = []#added by raiber
        surv_br_arr = []#added by raiber
        loss_arr = []
        anlp_arr = []
        auc_prob = []
        death_prob = []#added by raiber
        auc_label = []
        cindex_label = []#added by raiber
        auc_time = [] # added by raiber, to get the batch time 
        m_time = [] # added by raiber
        b_time = [] # added by raiber
        #print self.test_data_win.size + self.test_data_lose.size, "total size"
        total_time = 0
        for i in range(0, int(self.test_data_win.size / self.BATCH_SIZE)):
            test_batch_x, test_batch_y, test_batch_len, test_batch_market_price = self.test_data_win.next(
                self.BATCH_SIZE)
            start_time = time.time()
            bid_loss, bid_test_prob, anlp, single_time_survival, single_time_death = sess.run(
                [self.cost, self.predict, self.anlp_node, self.survival_rate_different_times, self.death_rate_different_times],
                feed_dict={self.tf_x: test_batch_x,
                           self.tf_y: test_batch_y,
                           self.tf_bid_len: test_batch_len,
                           self.tf_market_price:test_batch_market_price
                           })
            total_time += time.time() - start_time
            s_t_sr.append(np.array(single_time_survival))
            s_t_dr.append(np.array(single_time_death))
            auc_prob.append(np.array(bid_test_prob).T[0]) # bid_test_prob is the result from self.predict, and self.predict is stacked and tranposed
                                                          #array fron survival rates and deas rates, so when we do np.array(bid_test_prob).T[0]
                                                          # we get back only the survival rates 
            death_prob.append(np.array(bid_test_prob).T[1])
            auc_label.append(test_batch_y.T[0]) # because our label is either [0,1] or [1,0], we only care about the first index [0]
            cindex_label.append(test_batch_y.T[1])
            # changed by raiber from auc_label.append(test_batch_y.T[0]) to auc_label.append(test_batch_y.T[1])
            anlp_arr.append(anlp)
            loss_arr.append(bid_loss)
            auc_time.append(test_batch_market_price) # added by raiber
            m_time.append(test_batch_market_price)# added by raiber
            b_time.append(test_batch_len)# added by raiber
        mean_loss = np.array(loss_arr).mean()
        mean_anlp = np.array(anlp_arr).mean()
        log = self.getStatStr("TEST_WIN_DATA", self.global_step, 0.000001, mean_loss, mean_anlp)




        #result1, result2 = np.zeros([num_Event, len(EVAL_TIMES)]), np.zeros([num_Event, len(EVAL_TIMES)])
        #result1a, result2a = np.zeros([num_Event, len(EVAL_TIMES)]), np.zeros([num_Event, len(EVAL_TIMES)])
        #for t, t_time in enumerate(EVAL_TIMES):
        #eval_horizon = int(t_time)

        #if eval_horizon >= num_Category:
        #    print( 'ERROR: evaluation horizon is out of range')
        #    result1[:, t] = result2[:, t] = -1
        #else:
            # calculate F(t | x, Y, t >= t_M) = \sum_{t_M <= \tau < t} P(\tau | x, Y, \tau > t_M)
        #    risk = np.sum(pred[:,:,:(eval_horizon+1)], axis=2) #risk score until EVAL_TIMES
        #    for k in range(num_Event):
        #        result1a[k, t] = c_index(risk[:,k], te_time, (te_label[:,0] == k+1).astype(float), eval_horizon) #-1 for no event (not comparable)
        #        result2a[k, t] = brier_score(risk[:,k], te_time, (te_label[:,0] == k+1).astype(float), eval_horizon) #-1 for no event (not comparable)
        #        result1[k, t] = weighted_c_index(tr_time, (tr_label[:,0] == k+1).astype(int), risk[:,k], te_time, (te_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)
        #        result2[k, t] = weighted_brier_score(tr_time, (tr_label[:,0] == k+1).astype(int), risk[:,k], te_time, (te_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)

        #FINAL1[:, :, out_itr] = result1
        #FINAL2[:, :, out_itr] = result2


        
        print(log)
        for i in range(0, int(self.test_data_lose.size / self.BATCH_SIZE)):
            test_batch_x, test_batch_y, test_batch_len, test_batch_market_price = self.test_data_lose.next(
                self.BATCH_SIZE)
            bid_loss, bid_test_prob, anlp, single_time_survival, single_time_death= sess.run(
                                   [self.cost, self.predict,self.anlp_node, self.survival_rate_different_times, self.death_rate_different_times],
                                   feed_dict={self.tf_x: test_batch_x,
                                              self.tf_y: test_batch_y,
                                              self.tf_bid_len: test_batch_len,
                                              self.tf_market_price: test_batch_market_price
                                              })
            s_t_sr.append(np.array(single_time_survival))
            s_t_dr.append(np.array(single_time_death))
            auc_prob.append(np.array(bid_test_prob).T[0])
            death_prob.append(np.array(bid_test_prob).T[1])
            auc_label.append(test_batch_y.T[0])
            cindex_label.append(test_batch_y.T[1])
            # changed by raiber from auc_label.append(test_batch_y.T[0]) to auc_label.append(test_batch_y.T[1])
            anlp_arr.append(anlp)
            loss_arr.append(bid_loss)
            auc_time.append(test_batch_len) # added by raiber 
            m_time.append(test_batch_market_price)# added by raiber 
            b_time.append(test_batch_len)# added by raiber
        if len(auc_prob) > 0:
            #pdb.set_trace()
            try:  
                ll = np.array(s_t_sr).transpose(2,0,1).reshape(self.MAX_SEQ_LEN,-1)
                l_time = np.reshape(np.array(auc_time), [1, -1])[0]
                l_label = np.reshape(np.array(cindex_label), [1, -1])[0]
                df = pd.DataFrame(ll[:l_time.max() + 1,:]) # ll[:l_time.max() + 1,:]
                ev = EvalSurv(df, l_time, l_label, censor_surv='km')
                pyind = ev.concordance_td()
                time_grid = np.linspace(l_time.min(), l_time.max(), l_time.max())
                pybr= ev.integrated_brier_score(time_grid) 
                #y_true = ((Time_survival <= Time) * Death).astype(float)
                #arr[:, :2]
                auc = roc_auc_score(np.reshape(np.array(auc_label), [1, -1])[0],
                                np.reshape(np.array(auc_prob), [1, -1])[0])
                #cind_m_time_e = concordance_index(np.reshape(np.array(m_time), [1, -1])[0], 
                #                                np.reshape(np.array(auc_prob), [1, -1])[0], np.reshape(np.array(cindex_label), [1, -1])[0])
                #cind_b_time_e = concordance_index(np.reshape(np.array(b_time), [1, -1])[0], 
                #                                np.reshape(np.array(auc_prob), [1, -1])[0], np.reshape(np.array(cindex_label), [1, -1])[0])
                #cind_b_time_cen = concordance_index(np.reshape(np.array(b_time), [1, -1])[0], 
                #                                np.reshape(np.array(auc_prob), [1, -1])[0], np.reshape(np.array(auc_label), [1, -1])[0])
                #surv_label = np.reshape(np.array(cindex_label), [1, -1])[0] > 0
                #surv_c_ind1 = concordance_index_censored(surv_label, np.reshape(np.array(auc_time), [1, -1])[0], 
                #                np.reshape(np.array(death_prob), [1, -1])[0])
                #cind_auc_time_e = surv_c_ind1[0]
                
                #print("m_time_e:{0}, b_time_e:{1}, b_time_cen:{2}, auctime_e:{3}, ".format(cind_m_time_e, cind_b_time_e, cind_b_time_cen, cind_auc_time_e))
                
                #### using t
                #i_time = max(np.reshape(np.array(auc_time), [1, -1])[0]) - 1
                #r_test = np.core.records.fromarrays([surv_label, np.reshape(np.array(auc_time), [1, -1])[0]], names='cens,time')
                #bla, surv_brscore1 = brier_score(r_train, r_test, np.reshape(np.array(death_prob), [1, -1])[0], i_time)
                
                #surv_brscore = np.zeros([3])
                #bla, surv_brscore[0] = brier_score(r_train, r_test, np.reshape(np.array(death_prob), [1, -1])[0], 18)
                #bla, surv_brscore[1] = brier_score(r_train, r_test, np.reshape(np.array(death_prob), [1, -1])[0], 36)
                #bla, surv_brscore[2] = brier_score(r_train, r_test, np.reshape(np.array(death_prob), [1, -1])[0], i_time)
                
                #surv = np.mean(surv_brscore)
                #print("surv_brscore_auc_time:{0}".format(surv_brscore))
                
                #### using z
                #m_i_time = max(np.reshape(np.array(m_time), [1, -1])[0]) - 1
                #m_r_test = np.core.records.fromarrays([surv_label, np.reshape(np.array(m_time), [1, -1])[0]], names='cens,time')
                #m_bla, m_surv_brscore1 = brier_score(m_r_train,m_r_test, np.reshape(np.array(death_prob), [1, -1])[0], m_i_time)
                
                #m_surv_brscore = np.zeros([3])
                #m_bla, m_surv_brscore[0] = brier_score(m_r_train, m_r_test, np.reshape(np.array(death_prob), [1, -1])[0], 18)
                #m_bla, m_surv_brscore[1] = brier_score(m_r_train, m_r_test, np.reshape(np.array(death_prob), [1, -1])[0], 36)
                #m_bla, m_surv_brscore[2] = brier_score(m_r_train, m_r_test, np.reshape(np.array(death_prob), [1, -1])[0], 60)
                
                #m_surv = np.mean(m_surv_brscore)
                #print("surv_brscore_m_time:{0}".format(m_surv_brscore))
                
                
                ####### train 
                #tr_tt = np.reshape(np.array(tr_time), [1, -1])[0]
                #tr_m_tt = np.reshape(np.array(tr_m_time), [1, -1])[0]
                #tr_surv_label = np.reshape(np.array(tr_label), [1, -1])[0] #> 0
                
                #surv_brscore1 = weighted_brier_score(tr_tt, tr_surv_label, np.reshape(np.array(death_prob), [1, -1])[0], 
                #                     np.reshape(np.array(auc_time), [1, -1])[0], np.reshape(np.array(cindex_label), [1, -1])[0], 61)
                #m_surv_brscore1 = weighted_brier_score(tr_m_tt, tr_surv_label, np.reshape(np.array(death_prob), [1, -1])[0], 
                #                     np.reshape(np.array(m_time), [1, -1])[0], np.reshape(np.array(cindex_label), [1, -1])[0], 61)
                ##
                #idx_t = tr_tt < 52
                #tr_tt_a = tr_tt[idx]
                #tr_surv_label_a = tr_surv_label[idx]
                ##
                #r_train_n_a = np.core.records.fromarrays([tr_surv_label_a, tr_tt_a], names='cens,time')
                #r_train_n = np.core.records.fromarrays([tr_surv_label, tr_tt ], names='cens,time')
                ############
                #tt = np.reshape(np.array(auc_time), [1, -1])[0]
                #m_tt = np.reshape(np.array(m_time), [1, -1])[0]
                #i_time = max(tt) 
                #lm_time = max(m_tt) - 1 
                #m_i_time = min(max(m_tt),max(tr_tt)) + 1
                #idx = m_tt < m_i_time
                #m_tt_a = m_tt[idx]
                #surv_label_a = surv_label[idx]
                #r_test = np.core.records.fromarrays([surv_label, tt], names='cens,time')
                #m_r_test = np.core.records.fromarrays([surv_label_a, m_tt_a], names='cens,time')
                #bla, surv_brscore = brier_score(r_train, r_test, np.reshape(np.array(death_prob), [1, -1])[0], i_time)
                #m_bla, m_surv_brscore = brier_score(r_train, m_r_test, np.reshape(np.array(death_prob), [1, -1])[0], m_i_time)
                
                ####train
                #bla1, surv_brscore1 = brier_score(r_train_n, r_test, np.reshape(np.array(death_prob), [1, -1])[0], i_time)
                #m_bla1, m_surv_brscore1 = brier_score(r_train_n, m_r_test, np.reshape(np.array(death_prob), [1, -1])[0], m_i_time)
                ####
                
                #if eval_time is None:
                #    eval_time = [int(np.percentile(tr_time, 25)), int(np.percentile(tr_time, 50)), int(np.percentile(tr_time, 75))]
                ### VALIDATION  (based on average C-index of our interest)
                #eval_time = [18,36,60]

                ### EVALUATION
                #va_result = np.zeros([len(eval_time)])
                #va_result_m = np.zeros([len(eval_time)])
                #br_result = np.zeros([len(eval_time)])
                #br_result_m = np.zeros([len(eval_time)])
                #s_t_sr_array = np.array(s_t_sr)
                #s_t_dr_array = np.array(s_t_dr)
                #for t, t_time in enumerate(eval_time):
                #    eval_horizon = int(t_time)
                #    
                #    idx_observed = np.reshape(np.array(auc_time), [1, -1])[0] <= eval_horizon
                #    labels = np.reshape(np.array(cindex_label), [1, -1])[0] * idx_observed
                    #auc_ll = (labels < 1).astype(int)
                    #idx_observed_m = np.reshape(np.array(m_time), [1, -1])[0] <= eval_horizon
                    #labels_m = np.reshape(np.array(cindex_label), [1, -1])[0] * idx_observed_m 
                    
                #    surv_prob = np.prod(s_t_sr_array[:,:,:(eval_horizon+1)], axis=2) #survival score until eval_time
                #    risk_prob = np.prod(s_t_dr_array[:,:,:(eval_horizon+1)], axis=2)
                    #auc_result[t] = roc_auc_score(auc_ll,
                    #             np.reshape(np.array(surv_prob), [1, -1])[0])
                    #va_result[t] = concordance_index(np.reshape(np.array(auc_time), [1, -1])[0], 
                    #                            np.reshape(np.array(surv_prob), [1, -1])[0], labels)
                #    va_result[t] = c_index(np.reshape(np.array(risk_prob), [1, -1])[0], np.reshape(np.array(auc_time), [1, -1])[0],
                #            np.reshape(np.array(cindex_label), [1, -1])[0], eval_horizon)
                #    br_result[t] = weighted_brier_score(train_o[17].values, train_o[18].values, np.reshape(np.array(risk_prob), [1, -1])[0], 
                #                     np.reshape(np.array(auc_time), [1, -1])[0], np.reshape(np.array(cindex_label), [1, -1])[0], eval_horizon)
                
                    
                    #va_result1[0, t] = brier_score(risk[:,k], va_time, (va_label[:,0] == k+1).astype(int), eval_horizon)
                    #va_result1[k, t] = weighted_c_index(tr_time, (tr_label[:,0] == k+1).astype(int), risk[:,k], va_time, (va_label[:,0] == k+1).astype(int), eval_horizon)
                
                #c1 = np.mean(va_result)
                #print("hit_c_index:{0}".format(va_result))
                #print(va_result)
                #c2 = np.mean(br_result)
                #print("hit_br:{0}".format(br_result))
                #print(br_result)
                

                if auc >  self.max_valid:
                    self.stop_flag = 0
                    self.max_valid = auc 
                    name = 'test'
                    print( 'updated.... average c-index = ' + str('%.4f' %(auc)))
                    np.save('C:\\Users\\raibe\\Desktop\\Thesis Code\\DRSA\\results\\ndb_p\\10\\{}_s_t_sr'.format(name), np.array(s_t_sr))
                    np.save('C:\\Users\\raibe\\Desktop\\Thesis Code\\DRSA\\results\\ndb_p\\10\\{}_s_t_dr'.format(name), np.array(s_t_dr))
                    np.save('C:\\Users\\raibe\\Desktop\\Thesis Code\\DRSA\\results\\ndb_p\\10\\{}_cindex_label'.format(name), np.reshape(np.array(cindex_label), [1, -1])[0])
                    np.save('C:\\Users\\raibe\\Desktop\\Thesis Code\\DRSA\\results\\ndb_p\\10\\{}_auc_label'.format(name), np.reshape(np.array(auc_label), [1, -1])[0])
                    np.save('C:\\Users\\raibe\\Desktop\\Thesis Code\\DRSA\\results\\ndb_p\\10\\{}_auc_time'.format(name), np.reshape(np.array(auc_time), [1, -1])[0])
                    np.save('C:\\Users\\raibe\\Desktop\\Thesis Code\\DRSA\\results\\ndb_p\\10\\{}_auc_prob'.format(name), np.reshape(np.array(auc_prob), [1, -1])[0])
                    np.save('C:\\Users\\raibe\\Desktop\\Thesis Code\\DRSA\\results\\ndb_p\\10\\{}_death_prob'.format(name), np.reshape(np.array(death_prob), [1, -1])[0])
                    np.save('C:\\Users\\raibe\\Desktop\\Thesis Code\\DRSA\\results\\ndb_p\\10\\{}_m_time'.format(name), np.reshape(np.array(m_time), [1, -1])[0])
                    np.save('C:\\Users\\raibe\\Desktop\\Thesis Code\\DRSA\\results\\ndb_p\\10\\{}_b_time'.format(name), np.reshape(np.array(b_time), [1, -1])[0])
                    #self.save_model()
                else:
                    self.stop_flag += 1
                
                
                
                ############## final time specific evaluation 
                #EVAL_TIMES = [18,36,60]
                #for t, t_time in enumerate(EVAL_TIMES):
                #    eval_horizon = int(t_time)
                #    risk = np.prod(pred[:,:,:(eval_horizon+1)], axis=2) #risk score until EVAL_TIMES
                #    result1a[k, t] = c_index(risk[:,k], te_time, (te_label[:,0] == k+1).astype(float), eval_horizon) #-1 for no event (not comparable)
                #    result2a[k, t] = brier_score(risk[:,k], te_time, (te_label[:,0] == k+1).astype(float), eval_horizon) #-1 for no event (not comparable)
                #    result1[k, t] = weighted_c_index(tr_time, (tr_label[:,0] == k+1).astype(int), risk[:,k], te_time, (te_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)
                #    result2[k, t] = weighted_brier_score(tr_time, (tr_label[:,0] == k+1).astype(int), risk[:,k], te_time, (te_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)

                #FINAL1[:, :, out_itr] = result1
                #FINAL2[:, :, out_itr] = result2
                
                
                
                
                #surv_brscore = brier_score(survival_train, survival_test, estimate, times)
                #prob = np.reshape(np.array(auc_prob), [1, -1])[0]
                #risk = 1 - prob
                #eval_time = [20,40,60]
                #va_result1 = np.zeros([1, len(eval_time)]) 
                #for ind, t_time in enumerate(eval_time):
                #    eval_horizon = int(t_time)
                #    y_true = ((np.reshape(np.array(auc_time), [1, -1])[0] <= t_time) * np.reshape(np.array(auc_label), [1, -1])[0]).astype(float)
                #    va_result1[0, ind] = roc_auc_score(y_true, np.reshape(np.array(auc_prob), [1, -1])[0]) 
                #mean_C = (va_result1)/3
                #c1 = c_index(np.reshape(np.array(auc_prob), [1, -1])[0], np.reshape(np.array(auc_time), [1, -1])[0], np.reshape(np.array(auc_label), [1, -1])[0], 20)
                #c2 = c_index(np.reshape(np.array(auc_prob), [1, -1])[0], np.reshape(np.array(auc_time), [1, -1])[0], np.reshape(np.array(auc_label), [1, -1])[0], 40)
                #c3 = c_index(np.reshape(np.array(auc_prob), [1, -1])[0], np.reshape(np.array(auc_time), [1, -1])[0], np.reshape(np.array(auc_label), [1, -1])[0], 60)
                #mean_C = (c1+c2+c3)/3
                
                #b1 = brier_score(np.reshape(np.array(auc_prob), [1, -1])[0], np.reshape(np.array(auc_time), [1, -1])[0], np.reshape(np.array(auc_label), [1, -1])[0], 20)
                #b2 = brier_score(np.reshape(np.array(auc_prob), [1, -1])[0], np.reshape(np.array(auc_time), [1, -1])[0], np.reshape(np.array(auc_label), [1, -1])[0], 40)
                #b3 = brier_score(np.reshape(np.array(auc_prob), [1, -1])[0], np.reshape(np.array(auc_time), [1, -1])[0], np.reshape(np.array(auc_label), [1, -1])[0], 60)
                #mean_B = (b1+b2+b3)/3

                #eval_time = [20,40,60]
                #va_result1 = np.zeros([1, len(eval_time)]) 
                #for ind, t_time in enumerate(eval_time):
                #    eval_horizon = int(t_time)
                #    idx_observed = np.reshape(np.array(auc_time), [1, -1])[0] <= eval_horizon
                #    va_result1[0, ind] = roc_auc_score(np.reshape(np.array(auc_label), [1, -1])[0][idx_observed],
                #                  np.reshape(np.array(auc_prob), [1, -1])[0][idx_observed]) 
                
                
                #raiber return
                #eval_time = [20,40,60]
                #va_result1 = np.zeros([1, len(eval_time)]) 
                #for ind, t_time in enumerate(eval_time): #raiber: enumerate give us the index and the value , so t will be the index
                #     eval_horizon = int(t_time)
                #     idx_observed = np.reshape(np.array(auc_time), [1, -1])[0] <= eval_horizon
                     #print("t: {0}, p: {1}, e:{2}".format(t[idx_observed].shape,predicted_event_times[idx_observed].shape, e[idx_observed].shape))
                #     va_result1[0, ind] = concordance_index(event_times=(np.reshape(np.array(auc_time), [1, -1])[0][idx_observed]).tolist(), predicted_scores=(np.reshape(np.array(auc_prob), [1, -1])[0][idx_observed]).tolist(),
                #                            event_observed=(np.reshape(np.array(auc_label), [1, -1])[0][idx_observed]).tolist())
                #ci_index = np.mean(va_result1)
                # end raiber 
            except Exception:
                print("AUC ERROR")
                return
            #auc = np.mean(va_result1)
            auc_arr.append(auc)
            lc_arr.append(pyind)
            #surv_cind_arr.append(cind_auc_time_e)#added by raiber
            br_arr.append(pybr)#added by raiber
            #m_br_arr.append(surv_brscore1)#added by raiber
            ############
            #br_arr1.append(surv_brscore1)#added by raiber
            #m_br_arr1.append(m_surv_brscore1)#added by raiber
            ###########
        mean_loss = np.array(loss_arr).mean()
        mean_auc = np.array(auc_arr).mean()
        #mean_c_surv = np.array(surv_cind_arr).mean()
        mean_lc = np.array(lc_arr).mean()
        mean_br = np.array(br_arr).mean() #added by raiber
        #mean_br_m = np.array(m_br_arr).mean() #added by raiber
        ###########
        #mean_br1 = np.array(br_arr1).mean() #added by raiber
        #mean_br_m1 = np.array(m_br_arr1).mean() #added by raiber
        ############
        mean_anlp = np.array(anlp_arr).mean() 
        #mean_br, mean_br_m
        log = self.getStatStr_test("TEST", self.global_step, mean_auc, mean_lc, mean_br, mean_loss, mean_anlp)
        self.force_write(log)
        print(log)
        return mean_auc, mean_loss, mean_anlp

    def force_write(self, log):
        if not self.SAVE_LOG:
            return
        self.train_log_txt = open(self.train_log_txt_filename, 'a')
        self.train_log_txt.write(log)
        self.train_log_txt.close()


In [7]:
#from BASE_MODEL import BASE_RNN
import sys

#default parameter
FEATURE_SIZE = 26 # dataset input fields countnn

MAX_DEN = 912736 # max input data demension 97419, 2047294, 2047341, here is the max the embedding can get 
EMB_DIM = 32
BATCH_SIZE = 102
# max time for mortgage data 93
MAX_SEQ_LEN = int(146) # Dimension of time horizon of interest (equivalent to the output dimension per survival)
TRAING_STEPS = 10000000 # changed by raiber from 10000000

STATE_SIZE = 128
GRAD_CLIP = 5.0
L2_NORM = 0.001
ADD_TIME = True
ALPHA = 1.2 # coefficient for cross entropy
BETA = 0.2 # coefficient for anlp
input_file="2259" #toy dataset

if len(sys.argv) < 2:
    print ("Please input learning rate. ex. 0.0001")
    sys.exit(0)

LR = float(0.0001)
LR_ANLP = LR
RUNNING_MODEL = BASE_RNN(EMB_DIM=EMB_DIM,
                         FEATURE_SIZE=FEATURE_SIZE,
                         BATCH_SIZE=BATCH_SIZE,
                         MAX_DEN=MAX_DEN,
                         MAX_SEQ_LEN=MAX_SEQ_LEN,
                         TRAING_STEPS=TRAING_STEPS,
                         STATE_SIZE=STATE_SIZE,
                         LR=LR,
                         GRAD_CLIP=GRAD_CLIP,
                         L2_NORM=L2_NORM,
                         INPUT_FILE=input_file,
                         ALPHA=ALPHA,
                         BETA=BETA,
                         ADD_TIME_FEATURE=ADD_TIME,
                         FIND_PARAMETER=False,
                         ANLP_LR=LR,
                         DNN_MODEL=False,
                         DISCOUNT=1,
                         ONLY_TRAIN_ANLP=False,
                         LOG_PREFIX="drsa")
RUNNING_MODEL.create_graph()
RUNNING_MODEL.run_model()

drsa32_102_128_0.000100_0.000100_0.001000_2259_1.20_0.20_True_False_1_1 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().
data size  7579 

data size  8000 

data size  1898 

data size  102 

2259	TRAIN	100	1.536856	0.4046	7.0954	3.263332	102	128	0.000100	0.000100	0.001	1.2	0.2

2259	TEST_WIN_DATA	100

2259	TRAIN	1900	0.100242	0.9982	2.8654	0.693432	102	128	0.000100	0.000100	0.001	1.2	0.2

2259	TEST_WIN_DATA	1900	0.220845	0.0000	4.0649	1.078032	102	128	0.000100	0.000100	0.001	1.2	0.2

updated.... average c-index = 0.8972
2259	TEST	1900	0.282367	0.8972	0.8509	0.2255	4.2471	1.188332	102	128	0.000100	0.000100	0.001	1.2	0.2

2259	TRAIN	2000	0.089990	0.9973	2.7274	0.653532	102	128	0.000100	0.000100	0.001	1.2	0.2

2259	TEST_WIN_DATA	2000	0.201231	0.0000	4.1176	1.065032	102	128	0.000100	0.000100	0.001	1.2	0.2

2259	TEST	2000	0.277254	0.8849	0.8530	0.2242	4.3361	1.199932	102	128	0.000100	0.000100	0.001	1.2	0.2

2259	TRAIN	2100	0.080137	0.9984	2.6223	0.620632	102	128	0.000100	0.000100	0.001	1.2	0.2

2259	TEST_WIN_DATA	2100	0.184788	0.0000	4.2990	1.081532	102	128	0.000100	0.000100	0.001	1.2	0.2

2259	TEST	2100	0.264392	0.8916	0.8541	0.2221	4.5475	1.226832	102	128	0.000100	0.000100	0.001	1.2	0.2

2259	TRAIN	2200	0.071157	0.9994	2.6307	0.611532	102	128	0.000100	0.000100	0.001	1.2	0.2

2259	TEST_